## Segments

Action Sets? `./SET/*.BIN` and `SET.MUK`

## action sets

describe which set of sprites refer to a certain action, game files has a couple of action set BIN files, the rest are merged into one big SET.MUK file

Action files for regular sprites are laid out in the following format:
```
ACTION_FRAME_COUNT   POSITION_FORMAT
X_COORD Y_COORD CEL_NUMBER FLIP TIME_IN_TICKS {PAGE_NUMBER}
X_COORD Y_COORD CEL_NUMBER FLIP TIME_IN_TICKS {PAGE_NUMBER}
```
—and so on, where each new line is a new frame.
       Notes
    • POSITION_FORMAT can be R or A; for sprites, however, it is seemingly always set to R.
    • FLIP must be either 1 or -1.
    
## for large composite sprites:

Action files for boss sprites are laid out in the following format:
```
ACTION_FRAME_COUNT   POSITION_FORMAT   SPRITE_COUNT
PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}
PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}
```
—and so on, where each new line is a new frame.
       Notes
    • For boss sprites, POSITION_FORMAT is B.
    • FLIP must be either 1 or -1.
    
A frame is now defined by multiple parts, where frames are delimited by a new zero part.
For parts other than a zero part (that is, the start of a new frame), the x and y values are the deltas (changes) relative to the zero part that define where this part is placed.
A Damaged version is specified by a 1, which defaults to zero if not damaged.
Huh? What does that (above) mean?
TIME_IN_TICKS is irrelevant for cels beyond the first one in the frame; however, you must enter it as a place holder if specifying the part is a Damaged one or the cel is on a different page. The same applies to the Damaged flag if specifying a different page number.
ACTION_FRAME_COUNT is the number of actual frames, not the total number of lines in the file.
If there is a Damaged cel version, it must be listed before the normal cel or MAKESET will issue an error.
### Coding Events
Events are actions such as sprite shooting or making contact with a sword. They are coded in as a line in the Action files by setting the CEL_NUMBER value to -1 and the ticks to zero.

In [ ]:
# on windows 1 second is 10,000 ticks

In [ ]:
import os
from collections import defaultdict
import logging
from IPython.display import HTML, display
import math
from itertools import islice, product
from enum import Enum
from PIL import Image

In [ ]:
# tilesets; DICT[str, Tileset]
%run ./CEL\ INSPECTOR.ipynb

In [ ]:
def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))


def format_word(v, w):
    text = '{:02X} {:02X}'.format(v, w)
    return f'{text}'

    
def hex_view(data):
    text = []
    text.append(
        14 * '&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8)]) +
        '&nbsp;&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8, 16)])
    
    )
    text.append('')
    for i, x in enumerate(chunk(data, 16)):
        text.append('{:08X}:&nbsp;&nbsp;&nbsp;&nbsp;'.format(i * 16) +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[:8:2], x[1:8:2])]) +
              '&nbsp;&nbsp;' +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[8:16:2], x[9:16:2])])
             )
    return '<span style="font-family: Courier New, Courier, Lucida Sans Typewriter, Lucida Typewriter, monospace;">{}</span>'.format("<br>".join(text))


def display_hex_view(data):
    display(HTML(hex_view(data)))


#display_hex_view(bytes(range(132)))

In [ ]:
# Look at the top of the files
FOLDER = '/home/alex/.wine/drive_c/GOG Games/WH40K Final Liberation/'
OUTPUT = './sprites/'

for filepath in sorted(os.listdir(FOLDER)):
    filesize = os.path.getsize(FOLDER + '/' + filepath)
    file= filepath.split('.')
    if len(file) == 2 and file[1] == 'MUK':
        print(file[0], filesize)

In [ ]:
from itertools import zip_longest

Frame = namedtuple(
    'Frame', 
    [
        'page_number', 'cel_number', 'time',
        'flip', 'y', 'x', 'part', 'unknown'
    ])


def read_virtual_file(file, filepath, end_pos):
    sprite_number = filepath.split('.')[0].replace('SET', '')
    header = file.read(0x28)
    unknown1 = int.from_bytes(file.read(2), 'little', signed=True)
    headers = int.from_bytes(file.read(2), 'little', signed=True)
    # print("sets:", headers)
    unknown3 = int.from_bytes(file.read(2), 'little', signed=True)
    h = []
    for i in range(headers):
        # at least part of the second value is an index?
        h.append([
            int.from_bytes(file.read(4), 'little', signed=True),
            int.from_bytes(file.read(4), 'little', signed=True)
        ])
    prev = 0
    header_size = 0x28 + 6 + 8 * headers

    if headers > 0:
        try:  # not all units have a CEL (e.g. captain units)
            base_sprite = Sprite(f'PAGE{sprite_number}.CEL')
        except:
            base_sprite = None
        other_sprites = {}

    for set_index, (start, end) in enumerate(zip_longest(h[0:], h[1:])):
        if end is None:
            end = [os.fstat(file.fileno()).st_size]

        set_frames = int.from_bytes(file.read(2), 'little')
        frames = []
        durations = []
        for i in range(set_frames):
            if i == 0:
                start_seg = file.read(2)
                #assert int.from_bytes(start_seg, 'little') == 256, f"Invalid start of set {start_seg}"

            frame_data = Frame(*[
                int.from_bytes(x, 'little', signed=True) 
                for x in chunk(file.read(16), 2)
            ])
            #assert frame_data.flip in (-1, 1), f"Flip value is invalid {frame_data.flip}"  # not sure this is flip?
            #assert frame_data.unknown == 0, f"unknown value is {frame_data.unknown}"  # not always zero
            
            # why does the warhound have a negative page id for it's weapons?
            # damage indidcator??
            if frame_data.time > 0:
                try:
                    if frame_data.page_number == int(sprite_number):
                        im = base_sprite.images[abs(frame_data.cel_number) % len(base_sprite.images)] if frame_data.flip == 1 else base_sprite.images[frame_data.cel_number].transpose(Image.FLIP_LEFT_RIGHT)
                    else:
                        temp_sprite = other_sprites.get(
                            abs(frame_data.page_number), Sprite(f'PAGE{abs(frame_data.page_number):04}.CEL')
                        )
                        im = temp_sprite.images[abs(frame_data.cel_number) % len(temp_sprite.images)] if frame_data.flip == 1 else temp_sprite.images[frame_data.cel_number].transpose(Image.FLIP_LEFT_RIGHT)

                    if frame_data.part == 0:
                        frames.append(im)
                        durations.append(frame_data.time)
                    else:
                        if len(frames) > 0:
                            frames[-1].paste(im, (frame_data.x, frame_data.y), mask=im)
                        else:
                            frames.append(im)
                except KeyError as e:
                    print("Invalid CEL index {} of {}\n{}".format(
                        frame_data[1], 'x', frame_data)
                    )
            print(frame_data)
        if not os.path.exists(f'gifs/{sprite_number}'):
            os.makedirs(f'gifs/{sprite_number}')
        if len(frames) > 0:
            frames[0].save(f'./gifs/{sprite_number}/{sprite_number}-{set_index}.gif', format='GIF',
               append_images=frames[1:], save_all=True, duration=durations, loop=0)
        display_gif(f'./gifs/{sprite_number}/{sprite_number}-{set_index}.gif')
        #assert end_pos == file.tell(), f"end position is {file.tell()} and not {end_pos}"

In [ ]:
def display_gif(fn):
    return display(HTML('<img src="{}">'.format(fn)))

In [ ]:
from collections import namedtuple 
      
File = namedtuple('File', ['name', 'offset', 'length'])


# a muk looks to be a bit like a tar file, to group files up
def read_muk():
    with open(f'{FOLDER}/SET.MUK', 'rb') as file:
        header = file.read(34)
        _ = file.read(4)
        unknown1 = int.from_bytes(file.read(2), 'little')
        packaged_files = int.from_bytes(file.read(2), 'little')
        unknown2 = int.from_bytes(file.read(2), 'little')
        unknown3 = int.from_bytes(file.read(2), 'little')
        unknown4 = file.read(10)
        #print(packaged_files, unknown2, unknown3, unknown4)

        # next block contains filenames, offests and their lengths in the final block
        files = []
        for i in range(packaged_files):
            filename = file.read(11).decode('utf-8')
            blank = file.read(2) # maybe double blank is the terminator?
            file_offset = int.from_bytes(file.read(4), 'little')
            file_length = int.from_bytes(file.read(4), 'little')
            files.append(File(filename, file_offset, file_length))

        # use the preceeding info to jump around the file, to read the files tared up
        for f in files:
            file.seek(f.offset)
            print(f.name, f.offset, f.length)
            try:
                read_virtual_file(file, f.name, f.offset + f.length)
            except Exception as e:
                logging.warning(e)
        pos, size = file.tell(), os.fstat(file.fileno()).st_size
        assert pos == size , f"Did not read to EOF {pos} of {size}"


read_muk()

In [ ]:
# check header data for meta data for set?
# glich in verticle missle
# use thud gun 1208 to debug parts and offsets
#check negative pages, are they abs(value) or just the remaining bits? apart from top bit
# thing final value is z value